*Accompanying code examples of the book "Introduction to Artificial Neural Networks and Deep Learning: A Practical Guide with Applications in Python" by [Sebastian Raschka](https://sebastianraschka.com). All code examples are released under the [MIT license](https://github.com/rasbt/deep-learning-book/blob/master/LICENSE). If you find this content useful, please consider supporting the work by buying a [copy of the book](https://leanpub.com/ann-and-deeplearning).*
  
Other code examples and content are available on [GitHub](https://github.com/rasbt/deep-learning-book). The PDF and ebook versions of the book are available through [Leanpub](https://leanpub.com/ann-and-deeplearning).

In [1]:
%load_ext watermark
%watermark -a 'Sebastian Raschka' -v -p torch

Sebastian Raschka 

CPython 3.6.3
IPython 6.2.1

torch 0.3.0.post4


# Model Zoo -- Softmax Regression

Implementation of softmax regression (multinomial logistic regression).

In [1]:
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
import torch


##########################
### SETTINGS
##########################

# Hyperparameters
learning_rate = 0.001
num_epochs = 10
batch_size = 256

# Architecture
num_features = 784
num_classes = 10


##########################
### MNIST DATASET
##########################

train_dataset = datasets.MNIST(root='data', 
                               train=True, 
                               transform=transforms.ToTensor(),  
                               download=True)

test_dataset = datasets.MNIST(root='data', 
                              train=False, 
                              transform=transforms.ToTensor())


train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=batch_size, 
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size, 
                         shuffle=False)


# Checking the dataset
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

Image batch dimensions: torch.Size([256, 1, 28, 28])
Image label dimensions: torch.Size([256])


In [2]:
##########################
### MODEL
##########################

class SoftmaxRegression(torch.nn.Module):

    def __init__(self, num_features, num_classes):
        super(SoftmaxRegression, self).__init__()
        self.linear = torch.nn.Linear(num_features, num_classes)
        self.linear.weight.data.zero_()
        self.linear.bias.data.zero_()
        
    def forward(self, x):
        logits = self.linear(x)
        probas = F.softmax(logits, dim=1)
        return logits, probas

model = SoftmaxRegression(num_features=num_features,
                          num_classes=num_classes)

if torch.cuda.is_available():
    model.cuda()
    
    

##########################
### COST AND OPTIMIZER
##########################

# note that the PyTorch implementation of
# CrossEntropyLoss works with logits, not
# probabilities
cost_fn = torch.nn.CrossEntropyLoss()  

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [4]:
from torch.autograd import Variable
import torch.nn.functional as F

torch.manual_seed(0)


def compute_accuracy(model, data_loader):
    correct_pred, num_examples = 0, 0
    for features, targets in data_loader:
        features = Variable(features.view(-1, 28*28))
        if torch.cuda.is_available():
            features.cuda()
        logits, probas = model(features)
        _, predicted_labels = torch.max(probas.data, 1)
        num_examples += targets.size(0)
        correct_pred += (predicted_labels.cpu() == targets).sum()
    return correct_pred/num_examples * 100
    

for epoch in range(num_epochs):
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = Variable(features.view(-1, 28*28))
        targets = Variable(targets)
        
        if torch.cuda.is_available():
            features, targets = features.cuda(), targets.cuda()
            
        ### FORWARD AND BACK PROP
        logits, probas = model(features)
        cost = cost_fn(logits, targets)
        optimizer.zero_grad()
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f' 
                   %(epoch+1, num_epochs, batch_idx, 
                     len(train_dataset)//batch_size, cost.data[0]))
            
    print('Epoch: %03d/%03d training accuracy: %.2f%%' % (
          epoch+1, num_epochs, 
          compute_accuracy(model, train_loader)))

Epoch: 001/010 | Batch 000/234 | Cost: 2.3026
Epoch: 001/010 | Batch 050/234 | Cost: 0.5400
Epoch: 001/010 | Batch 100/234 | Cost: 0.3796
Epoch: 001/010 | Batch 150/234 | Cost: 0.4273
Epoch: 001/010 | Batch 200/234 | Cost: 0.3295
Epoch: 001/010 training accuracy: 89.91%
Epoch: 002/010 | Batch 000/234 | Cost: 0.4648
Epoch: 002/010 | Batch 050/234 | Cost: 0.3611
Epoch: 002/010 | Batch 100/234 | Cost: 0.3480
Epoch: 002/010 | Batch 150/234 | Cost: 0.3730
Epoch: 002/010 | Batch 200/234 | Cost: 0.2631
Epoch: 002/010 training accuracy: 90.76%
Epoch: 003/010 | Batch 000/234 | Cost: 0.3017
Epoch: 003/010 | Batch 050/234 | Cost: 0.4146
Epoch: 003/010 | Batch 100/234 | Cost: 0.3101
Epoch: 003/010 | Batch 150/234 | Cost: 0.2404
Epoch: 003/010 | Batch 200/234 | Cost: 0.3169
Epoch: 003/010 training accuracy: 91.43%
Epoch: 004/010 | Batch 000/234 | Cost: 0.2980
Epoch: 004/010 | Batch 050/234 | Cost: 0.3591
Epoch: 004/010 | Batch 100/234 | Cost: 0.2404
Epoch: 004/010 | Batch 150/234 | Cost: 0.2611
Epo

In [5]:
print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader)))

Test accuracy: 92.33%
